In [1]:
include("../LiPoSID.jl")

include("CtrlSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

@polyvar γ[1:3]
@polyvar a[1:3]

Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]+0.0
           im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

κ₁ = -γ[1]+γ[2]+γ[3]
κ₂ =  γ[1]-γ[2]+γ[3]
κ₃ =  γ[1]+γ[2]-γ[3]

constr1 = κ₁ + κ₂ + κ₃  
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - a[1]^2 - a[2]^2 - a[3]^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2 


#constraints = [κ₁, κ₂, κ₃, γ[1], γ[2], γ[3], constr1, constr2, constr3]

constraints = [κ₁, κ₂, κ₃, γ[1], γ[2], γ[3], constr2, constr3]

@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

H0ˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

H0ᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(H0ᴷˢʸᵐᵇ) == 0
@assert H0ᴷˢʸᵐᵇ == H0ˢʸᵐᵇ

Hˢʸᵐᵇ = H0ˢʸᵐᵇ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.5+0.0im)ϵ                       (0.5+0.0im)h_Re + (0.0+0.5im)h_Im
 (0.5+0.0im)h_Re + (0.0-0.5im)h_Im  (-0.5+0.0im)ϵ

In [18]:
Cˢʸᵐᵇ

3×3 Matrix{Polynomial{true, ComplexF64}}:
 -γ₁ + γ₂ + γ₃  (0.0-1.0im)a₃  (0.0+1.0im)a₂
 (0.0+1.0im)a₃  γ₁ + -γ₂ + γ₃  (0.0-1.0im)a₁
 (0.0-1.0im)a₂  (0.0+1.0im)a₁  γ₁ + γ₂ + -γ₃

In [3]:
constraints

8-element Vector{Polynomial{true, Int64}}:
 -γ₁ + γ₂ + γ₃
 γ₁ - γ₂ + γ₃
 γ₁ + γ₂ - γ₃
 γ₁
 γ₂
 γ₃
 -γ₁² + 2γ₁γ₂ + 2γ₁γ₃ - γ₂² + 2γ₂γ₃ - γ₃² - a₁² - a₂² - a₃²
 -γ₁³ + γ₁²γ₂ + γ₁²γ₃ + γ₁γ₂² - 2γ₁γ₂γ₃ + γ₁γ₃² + γ₁a₁² - γ₁a₂² - γ₁a₃² - γ₂³ + γ₂²γ₃ + γ₂γ₃² - γ₂a₁² + γ₂a₂² - γ₂a₃² - γ₃³ - γ₃a₁² - γ₃a₂² + γ₃a₃²

In [ ]:
function det_2x2(matrix)
    # Ensure the matrix is 2x2
    if size(matrix) != (2, 2)
        throw(ArgumentError("Matrix must be 2x2"))
    end

    # Calculate the determinant
    return matrix[1, 1] * matrix[2, 2] - matrix[1, 2] * matrix[2, 1]
end

function infidelity_constr(ρ, σ, d)
    f = 1-d
    return 4*(det_2x2(ρ)*det_2x2(σ)) - f^2 + 2*f*tr(ρ * σ) - tr(ρ * σ)^2
end

function infidelity_norm(ρ, σ)
    (1-4*(det_2x2(ρ)*det_2x2(σ))-2*tr(ρ * σ) + tr(ρ * σ)^2)*(1 -(1-tr(ρ * σ))+(1-tr(ρ * σ))^2)#-(1-tr(ρ * σ))^3)
end


function kossak_obj(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

function kossak_obj_infidelity_constr(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    constr = 0
    obj = 0
    δ = 1e-6
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i],t[i]) - 4*Dc(ρ[i-1],t[i-1]) - Dc(ρ[i-2],t[i-2])) / 3
        obj += LiPoSID.frobenius_norm2(ρ1-ρ2)
        constr += (infidelity_constr(ρ1, ρ2, δ))^2
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    if isempty(monomials(constr))
        constr = 0. 
    else
        constr = sum(real(coef) * mon for (coef, mon) in zip(coefficients(constr), monomials(constr)))
    end

    return obj, constr

end


function kossak_obj_pseudo_fidelity(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += pseudo_fidelity_norm(
            ρ[i],
            ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

function kossak_obj_infidelity(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += infidelity_norm(
            ρ[i],
            ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )^2
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

In [44]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

function kossak_GEXY_obj_infid(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj_infidelity(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ); print("G")
    polyE   = kossak_obj_infidelity(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ); print("E")
    polyX  = kossak_obj_infidelity(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ); print("X")
    polyY  = kossak_obj_infidelity(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ); println("Y")

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY 
end


function kossak_GEXY_obj_infid_constr(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG, constrG = kossak_obj_infidelity_constr(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE, constrE  = kossak_obj_infidelity_constr(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX, constrX  = kossak_obj_infidelity_constr(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY, constrY  = kossak_obj_infidelity_constr(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY
    constrGEXY = constrG + constrE + constrX + constrY

    return polyGEXY, constrGEXY 
end


function kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj_pseudo_fidelity(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj_pseudo_fidelity(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj_pseudo_fidelity(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj_pseudo_fidelity(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_obj_pseudo_fidelity (generic function with 1 method)

In [6]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γᵢ = "0.079477"

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)

((Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.08666666666

In [90]:
polyGEXY = kossak_GEXY_obj_infid(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

GEXY
minimum(abs.(coefficients(polyGEXY))) = 1.8774283812259205e-17
maximum(abs.(coefficients(polyGEXY))) = 5.4879696036331


5.4879696036331

In [81]:
polyGEXY = kossak_GEXY_obj_infid(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

GEXY
minimum(abs.(coefficients(polyGEXY))) = 3.879592866409097e-13
maximum(abs.(coefficients(polyGEXY))) = 5.498958359900041


5.498958359900041

In [68]:
polyGEXY = kossak_GEXY_obj_infid(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

GEXY
minimum(abs.(coefficients(polyGEXY))) = 1.1552913938042863e-7
maximum(abs.(coefficients(polyGEXY))) = 6.116604301161447


6.116604301161447

In [45]:
polyGEXY = kossak_GEXY_obj_infid(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

GEXY
minimum(abs.(coefficients(polyGEXY))) = 5.1211899117138e-14
maximum(abs.(coefficients(polyGEXY))) = 0.01874482996856581


0.01874482996856581

In [37]:
polyGEXY = kossak_GEXY_obj_infid(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

GEXY
minimum(abs.(coefficients(polyGEXY))) = 1.1552913938042863e-7
maximum(abs.(coefficients(polyGEXY))) = 6.116604301161447


6.116604301161447

In [91]:
optu,solu,datau = tssos_first(polyGEXY, variables(polyGEXY), TS="block", solution=true)

*********************************** TSSOS ***********************************
TSSOS is launching...


InexactError: InexactError: Int64(2.5)

In [55]:
optu,solu,datau = tssos_first(polyGEXY, variables(polyGEXY), TS="block", solution=true)

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[55]
[1]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 55.
Assembling the SDP...
There are 715 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 715             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2 (scalarized: 1595)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency ch

(4.550037754638908e-7, [-0.08105769035691421, 0.32647918189727143, 0.2468202182652219, 0.0005192398058168271, 0.0002567284300347723, -0.2578032512364387, 24.75360001145135, 0.0009254666906926178, -0.0009025043551930306], TSSOS.upop_data(9, 0, PolyVar{true}[γ₁, γ₂, γ₃, a₁, a₂, a₃, ϵ, h_Re, h_Im], 2.311366814059853e-7γ₁⁴ + 1.0522744114986488e-7γ₁³γ₂ + 1.496924956660305e-7γ₁³γ₃ - 1.7529413746468977e-8γ₁³a₁ - 4.798572450893505e-9γ₁³a₂ + 3.4704982577158606e-7γ₁³a₃ - 1.4993108861311204e-7γ₁³ϵ - 9.781438289765226e-11γ₁³h_Re + 5.619261804418164e-11γ₁³h_Im + 1.1043907433307101e-7γ₁²γ₂² + 7.205482411029943e-8γ₁²γ₂γ₃ - 9.820325670474638e-9γ₁²γ₂a₁ - 6.640324872911815e-9γ₁²γ₂a₂ + 1.713195491859262e-7γ₁²γ₂a₃ - 6.319225758277702e-8γ₁²γ₂ϵ - 5.347217911859134e-11γ₁²γ₂h_Re + 2.986559983779324e-11γ₁²γ₂h_Im + 2.764443624046994e-7γ₁²γ₃² - 2.4085023402602117e-11γ₁²γ₃a₁ + 3.975441277910585e-11γ₁²γ₃a₂ + 8.601856364677223e-7γ₁²γ₃a₃ - 7.501120178916008e-8γ₁²γ₃ϵ - 1.6307637370247273e-12γ₁²γ₃h_Re + 1.265308474419

In [83]:
solutionu = variables(polyGEXY) => solu

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solutionu)
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solutionu)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.503248+0.0im)   (-0.0+0.251061im)
 (-0.0-0.251061im)  (0.000362935+0.0im)

In [93]:
opt,sol,data = cs_tssos_first([polyGEXY, constraints...], variables(polyGEXY), 4, solution=true)#, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9]
[1]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.000522173 seconds. The maximal size of cliques is 9.
Starting to compute the block structure...
Obtained the block structure in 0.164351272 seconds.
The maximal size of blocks is 715.
Assembling the SDP...
There are 24310 affine constraints.
SDP assembling time: 0.729130296 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 24310           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1      

(1.0939279566941167, [0.09854608038241999, 0.16457074850058304, 0.2629789798030374, 1.7060967898490654e-6, -1.281669530326873e-6, -0.2546975685814659, 25.145640511647958, 0.00046712162974767434, -0.00024718663483926876], TSSOS.mcpop_data(9, 0, 8, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0001, 0x0001, 0x0003], [0x0001, 0x0001, 0x0001, 0x0001, 0x0004], [0x0001, 0x0001, 0x0001, 0x0001, 0x0005]  …  [0x0008, 0x0008, 0x0009, 0x0009, 0x0009], [0x0008, 0x0009], [0x0008, 0x0009, 0x0009], [0x0008, 0x0009, 0x0009, 0x0009], [0x0008, 0x0009, 0x0009, 0x0009, 0x0009], [0x0009], [0x0009, 0x0009], [0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009, 0x0009]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001]], [[0x0002]], [[0x0003

In [94]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.329004+0.0im)   (-0.0+0.254698im)
 (-0.0-0.254698im)  (0.196954+0.0im)

In [95]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

Hˢⁱᵈ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (12.5728+0.0im)              (0.000233561-0.000123593im)
 (0.000233561+0.000123593im)  (-12.5728+0.0im)

In [49]:
subs(polyGEXY, solution)

2.872746631549239e-8

In [50]:
using BlackBoxOptim

In [96]:
res_cut = compare_optimizers(polyGEXY; SearchRange = (-260, 260),
 NumDimensions =length(variables(polyGEXY)), 
 MaxTime = 25.0);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, SimpleSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 6159 evals, 6045 steps, improv/step: 0.354 (last = 0.3543), fitness=-5050483.425158635
1.00 secs, 12969 evals, 12855 steps, improv/step: 0.356 (last = 0.3577), fitness=-5054314.154284848
1.50 secs, 19749 evals, 19635 steps, improv/step: 0.353 (last = 0.3476), fitness=-5054314.703397568
2.00 secs, 26470 evals, 26370 steps, improv/step: 0.354 (last = 0.3580), fitness=-5054314.703483351
2.50 secs, 33235 evals, 33844 steps, improv/step: 0.364 (last = 0.3972), fitness=-5054314.703483362
3.00 secs, 40020 evals, 41483 steps, improv/step: 0.398 (last = 0.5510), fitness=-5054314.703483362
3.50 secs, 46557 evals, 48954 steps, improv/step: 0.410 (last = 0.4748), fitness=-5054314.703483362
4.00 secs, 53271 evals, 56666 steps, improv/step: 0.418 (last = 0.4719), fitness=-5054314.703483362
4.50 

In [73]:
opt,sol,data = cs_tssos_first([polyGEXY, constraints...], variables(polyGEXY), 2, solution=true)#, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9]
[1]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.000102131 seconds. The maximal size of cliques is 9.
Starting to compute the block structure...
Obtained the block structure in 0.000385104 seconds.
The maximal size of blocks is 55.
Assembling the SDP...
There are 715 affine constraints.
SDP assembling time: 0.003118107 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 715             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 8         

(6.52709259134266e-5, [0.09785730105524452, 0.16485041461138092, 0.26256975856126324, 1.709724020988396e-6, -1.2276493998279938e-6, -0.25402152003760825, 25.145698462611552, 0.0004516505700446677, -0.00023828610430702766], TSSOS.mcpop_data(9, 0, 8, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0002], [0x0001, 0x0003], [0x0001, 0x0004], [0x0001, 0x0005], [0x0001, 0x0006], [0x0001, 0x0007], [0x0001, 0x0008]  …  [0x0006, 0x0009], [0x0007], [0x0007, 0x0007], [0x0007, 0x0008], [0x0007, 0x0009], [0x0008], [0x0008, 0x0008], [0x0008, 0x0009], [0x0009], [0x0009, 0x0009]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001]], [[0x0002]], [[0x0003]], [[0x0001, 0x0001], [0x0001, 0x0002], [0x0001, 0x0003], [0x0002, 0x0002], [0x0002, 0x0003], [0x0003, 0x0003], [0x0004, 0x0004], [0x0005, 0x0005], [0x0006, 0x0006]], [[0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0003], [0x0001, 0x0002, 0x0002], [0x00

In [75]:
solution = variables(polyGEXY) => sol

Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)
Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.329563+0.0im)   (-0.0+0.254022im)
 (-0.0-0.254022im)  (0.195577+0.0im)

In [30]:
polyGEXY, constrGEXY = kossak_GEXY_obj_infid_constr(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

minimum(abs.(coefficients(polyGEXY))) = 6.086837819355578e-38
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478


65.37495321955478

In [14]:
   
polyGEXY, constrGEXY = kossak_GEXY_obj_infid_constr(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

#polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

minimum(abs.(coefficients(polyGEXY))) = 6.086837819355578e-38
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478


65.37495321955478

In [22]:
constrGEXY
@show minimum(abs.(coefficients(constrGEXY)))
@show maximum(abs.(coefficients(constrGEXY)))

minimum(abs.(coefficients(constrGEXY))) = 7.733730835084247e-15
maximum(abs.(coefficients(constrGEXY))) = 0.08116270648482278


0.08116270648482278

In [9]:
length(variables(polyGEXY))

9

In [8]:
function filter_terms_by_relative_threshold(poly::Polynomial, relative_threshold::Float64)
    # Get all coefficients of the polynomial
    coeffs = coefficients(poly)
    
    # Find the largest coefficient by absolute value
    max_coeff = maximum(abs.(coeffs))
    
    # Calculate the effective threshold
    threshold = relative_threshold * max_coeff
    
    # Initialize an empty polynomial of the same type as the input
    new_poly = zero(poly)
    
    # Iterate over the terms and coefficients of the polynomial
    for (monomial, coeff) in zip(monomials(poly), coeffs)
        if abs(coeff) >= threshold
            new_poly += coeff * monomial
        end
    end
    
    return new_poly
end

# Example usage
@polyvar x y
p = 1e-13*x^2 + 2*x*y + 3*y^2 + 4e-14*y
relative_threshold = 1e-12
filtered_p = filter_terms_by_relative_threshold(p, relative_threshold)
println(filtered_p)

2.0*x*y + 3.0*y^2


In [10]:
using BlackBoxOptim

In [38]:
using TSSOS

In [23]:
opt,sol,data = cs_tssos_first([polyGEXY, constrGEXY, constraints...], variables(polyGEXY), 2, solution=true)#, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9]
[1]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.00020617 seconds. The maximal size of cliques is 9.
Starting to compute the block structure...
Obtained the block structure in 0.000484382 seconds.
The maximal size of blocks is 55.
Assembling the SDP...
There are 715 affine constraints.
SDP assembling time: 0.003203836 seconds.
Solving the SDP...
MOSEK warning 705 (MSK_RES_WRN_ZEROS_IN_SPARSE_ROW): 1 (nearly) zero elements are specified in sparse row ''(28) of matrix 'A'.
MOSEK warning 705 (MSK_RES_WRN_ZEROS_IN_SPARSE_ROW): 1 (nearly) zero elements are specified in sparse row ''(69) of matrix 'A'.
MOSEK warning 705 (MSK_RES_WRN_ZEROS_IN_SPARSE_ROW): 1 (nearly) zero elements are specified in sparse row ''(73) of 

(0.5159566958983086, [4.7809435967431545e-8, 4.012325203275053e-8, 7.45473721716024e-8, 1.0452200589589036e-6, 2.401075191374717e-6, 9.993004779952218e-5, -37.24497732787174, -0.0007091735339553856, 0.000286666107855685], TSSOS.mcpop_data(9, 0, 9, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0004], [0x0001, 0x0007], [0x0001, 0x0009], [0x0002], [0x0002, 0x0002], [0x0002, 0x0005], [0x0002, 0x0007]  …  [0x0006, 0x0009], [0x0007], [0x0007, 0x0007], [0x0007, 0x0008], [0x0007, 0x0009], [0x0008], [0x0008, 0x0008], [0x0008, 0x0009], [0x0009], [0x0009, 0x0009]], [[0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0001, 0x0003], [0x0001, 0x0001, 0x0001, 0x0004], [0x0001, 0x0001, 0x0001, 0x0005], [0x0001, 0x0001, 0x0001, 0x0006], [0x0001, 0x0001, 0x0001, 0x0007], [0x0001, 0x0001, 0x0001, 0x0008], [0x0001, 0x0001, 0x0001, 0x0009], [0x0001, 0x0001, 0x0002, 0x0002]  …  [0x0001], [0x0002], [0x0003], [0x0004], [0x0005], [0x0006], [0x0007], [

In [27]:
opt_i2, sol_i2, data_i2  = cs_tssos_higher!(data, solution=true, TS="MD")

Starting to compute the block structure...
No higher TS step of the CS-TSSOS hierarchy!


(nothing, nothing, TSSOS.mcpop_data(9, 0, 9, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0004], [0x0001, 0x0007], [0x0001, 0x0009], [0x0002], [0x0002, 0x0002], [0x0002, 0x0005], [0x0002, 0x0007]  …  [0x0006, 0x0009], [0x0007], [0x0007, 0x0007], [0x0007, 0x0008], [0x0007, 0x0009], [0x0008], [0x0008, 0x0008], [0x0008, 0x0009], [0x0009], [0x0009, 0x0009]], [[0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0001, 0x0003], [0x0001, 0x0001, 0x0001, 0x0004], [0x0001, 0x0001, 0x0001, 0x0005], [0x0001, 0x0001, 0x0001, 0x0006], [0x0001, 0x0001, 0x0001, 0x0007], [0x0001, 0x0001, 0x0001, 0x0008], [0x0001, 0x0001, 0x0001, 0x0009], [0x0001, 0x0001, 0x0002, 0x0002]  …  [0x0001], [0x0002], [0x0003], [0x0004], [0x0005], [0x0006], [0x0007], [0x0008], [0x0009], []], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001]], [[0x0002]], [[0x0003]], [[0x0001, 0x0001], [0x0001, 0x0002], [0x0001, 0

In [24]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (6.68612e-8+0.0im)  (0.0-9.993e-5im)
 (0.0+9.993e-5im)    (8.22336e-8+0.0im)

In [25]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

Hˢⁱᵈ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-18.6225-0.0im)              (-0.000354587+0.000143333im)
 (-0.000354587-0.000143333im)  (18.6225-0.0im)

In [10]:
settings = mosek_para()
settings.tol_pfeas = 1e-9 # primal feasibility tolerance
settings.tol_dfeas = 1e-9 # dual feasibility tolerance
settings.tol_relgap = 1e-9 # relative primal-dual gap tolerance
settings.time_limit = 1e4 # limit of running time

10000.0

In [11]:
maxdegree(polyGEXY)

2

In [12]:
r4 = sum(variables(polyGEXY)[1:end-3].^4)

γ₁⁴ + γ₂⁴ + γ₃⁴ + a₁⁴ + a₂⁴ + a₃⁴

In [13]:
variables(polyGEXY)

9-element Vector{PolyVar{true}}:
 γ₁
 γ₂
 γ₃
 a₁
 a₂
 a₃
 ϵ
 h_Re
 h_Im

In [57]:
opt,sol,data = cs_tssos_first([polyGEXY,constraints...], variables(polyGEXY), 2, solution=true, mosek_setting=settings)

LoadError: MethodError: no method matching *(::Nothing, ::Vector{UInt16})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at ~/julia-1.7.3/share/julia/base/operators.jl:655
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at ~/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/matmul.jl:44
[0m  *([91m::StridedMatrix{var"#s31"} where var"#s31"<:MutableArithmetics.AbstractMutable[39m, ::StridedVector{var"#s37"} where var"#s37") at ~/.julia/packages/MutableArithmetics/iovKe/src/dispatch.jl:359
[0m  ...

In [15]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.357415+0.0im)   (-0.0+0.257095im)
 (-0.0-0.257095im)  (0.184727+0.0im)

In [43]:
Cˢⁱᵈ[2:3, 2:3]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.182096+0.0im)    (0.0-2.92046e-6im)
 (0.0+2.92046e-6im)  (0.000136974+0.0im)

In [26]:
opt1,sol1,data1 = cs_tssos_higher!(data, solution=true)

Starting to compute the block structure...
Obtained the block structure in 0.002455888 seconds.
The maximal size of blocks is 45.
Assembling the SDP...
There are 615 affine constraints.
SDP assembling time: 0.016639253 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 615             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2               
  Matrix variables       : 11 (scalarized: 2079)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00       

(0.0002854067543135459, [0.09243120338768061, 0.17877539169851442, 0.27107081825440427, 2.9094816545861705e-6, 2.0840956763762985e-7, -0.25709451998041966, 25.14090334816911, 0.00027513778473787516, -0.000145458125791655], TSSOS.mcpop_data(9, 0, 6, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0004], [0x0001, 0x0007], [0x0001, 0x0009], [0x0002], [0x0002, 0x0002], [0x0002, 0x0005], [0x0002, 0x0007]  …  [0x0006, 0x0009], [0x0007], [0x0007, 0x0007], [0x0007, 0x0008], [0x0007, 0x0009], [0x0008], [0x0008, 0x0008], [0x0008, 0x0009], [0x0009], [0x0009, 0x0009]], [[0x0001]], [[0x0002]], [[0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001, 0x0001], [0x0001, 0x0002], [0x0001, 0x0003], [0x0002, 0x0002], [0x0002, 0x0003], [0x0003, 0x0003], [0x0004, 0x0004], [0x0005, 0x0005], [0x0006, 0x0006]], [[0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0002], [0x0001, 0x0001, 0x0003], [0x0001, 0x0002, 0x0002], [0x0001, 0x0002, 0x0003], [0x0001, 0x0003, 0x0003], [0x0001, 0x0004, 0x0004],

In [32]:
solution1

PolyVar{true}[γ₁, γ₂, γ₃, a₁, a₂, a₃, ϵ, h_Re, h_Im] => [0.09243120338768061, 0.17877539169851442, 0.27107081825440427, 2.9094816545861705e-6, 2.0840956763762985e-7, -0.25709451998041966, 25.14090334816911, 0.00027513778473787516, -0.000145458125791655]

In [27]:
solution1 = variables(polyGEXY) => sol1

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution1)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.357415+0.0im)   (-0.0+0.257095im)
 (-0.0-0.257095im)  (0.184727+0.0im)

In [30]:
polyGEXY_cut = filter_terms_by_relative_threshold(polyGEXY, 1e-50)
opt_cut,sol_cut,data_cut = tssos_first(polyGEXY_cut+1e-9*r4, variables(polyGEXY_cut), TS="block", solution=true, mosek_setting=settings) #, newton=false, TS="MD"

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[22, 1]
[1, 9]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 22.
Assembling the SDP...
There are 212 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 212             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 10              
  Matrix variables       : 2 (scalarized: 308)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependen

(6.704688336333514e-5, [-4.0138859724220594e-5, 0.2513588259247926, 0.25191609600158016, 0.000701275740401503, 0.00026134842152021956, -0.2511598128653184, 25.141303620288813, 0.0005396306938830951, -0.0008478540991835319], TSSOS.upop_data(9, 0, PolyVar{true}[γ₁, γ₂, γ₃, a₁, a₂, a₃, ϵ, h_Re, h_Im], 1.0e-9γ₁⁴ + 1.0e-9γ₂⁴ + 1.0e-9γ₃⁴ + 1.0e-9a₁⁴ + 1.0e-9a₂⁴ + 1.0e-9a₃⁴ + 0.05168674709213622γ₁² - 0.0008044950814777315γ₁a₁ + 0.0005179676236185148γ₁ϵ + 4.104630886766538e-5γ₁h_Im + 0.05172488613394287γ₂² - 0.0012450858989678402γ₂a₂ - 0.0005179676236185148γ₂ϵ + 3.451755329501592e-5γ₂h_Re + 1.0001847686681244γ₃² + 2.1131584754297306γ₃a₃ - 3.45175532950157e-5γ₃h_Re - 4.104630886766538e-5γ₃h_Im + 1.269688888901564a₁² - 8.005395128533376e-37a₁a₂ - 4.658513032068184e-37a₁a₃ + 0.0012450858989678402a₁ϵ + 2.1131584754297306a₁h_Im + 1.269688888901564a₂² + 4.647304221135261e-36a₂a₃ - 0.0008044950814777315a₂ϵ - 2.1131584754297306a₂h_Re + 1.269688888901564a₃² - 0.0012450858989678404a₃h_Re + 0.00080449508

In [45]:
solution_cut = variables(polyGEXY) => sol_cut

Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, solution_cut))

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{ComplexF64}:
 0.503315+0.0im              0.0+0.25116im
      0.0-0.25116im  0.000517131+0.0im

In [32]:
Cˢⁱᵈ[2:3, 2:3]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.000517131+0.0im)  (0.0-0.000701276im)
 (0.0+0.000701276im)  (-0.000597409+0.0im)

In [46]:
Hˢⁱᵈ = convert.(ComplexF64,subs(Hˢʸᵐᵇ, solution_cut))

2×2 Matrix{ComplexF64}:
     12.5707+0.0im          0.000269815-0.000423927im
 0.000269815+0.000423927im     -12.5707+0.0im

In [61]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

get_lindblad_operators (generic function with 1 method)

In [59]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [54]:
Canz = [  0.25133     im*0.25133  0
         -im*0.25133  0.25133     0
          0           0           0 ]

Cˢⁱᵈ = Canz 

3×3 Matrix{ComplexF64}:
 0.25133+0.0im          0.0+0.25133im  0.0+0.0im
     0.0-0.25133im  0.25133+0.0im      0.0+0.0im
     0.0+0.0im          0.0+0.0im      0.0+0.0im

In [87]:
Hˢⁱᵈ = convert.(ComplexF64,Hˢⁱᵈ)
Cˢⁱᵈ = convert.(ComplexF64,Cˢⁱᵈ)

3×3 Matrix{ComplexF64}:
 0.328705+0.0im              0.0+0.254448im           0.0-1.28417e-6im
      0.0-0.254448im    0.196746+0.0im                0.0-1.7044e-6im
      0.0+1.28417e-6im       0.0+1.7044e-6im  0.000137916+0.0im

In [88]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴷᴼᴺᴮ)
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    #Fˢᵖⁱⁿᵇᴼˢᴼⁿ= LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

    #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

    #h5open(tests_dir*tests_data_file_name,"cw") do fid
        #γ_group = open_group(fid, γᵢ) # open gamma coupling group
        #init_state_group = create_group(γ_group, state) # create initial state group
        #init_state_group["Fidelity_SB"] = convert.(Float64, Fˢᵖⁱⁿᵇᴼˢᴼⁿ)
        #init_state_group["F"] = convert.(Float64, F)
        #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
        #init_state_group["bloch_sid_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["bloch_sim_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        #init_state_group["time"] = tᵗˢᵗ
    #end
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Mimimal fidelity for 0.25133: 0.9960759512628733
Median fidelity for 0.25133: 0.9985765450612922


In [55]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴷᴼᴺᴮ)
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    #Fˢᵖⁱⁿᵇᴼˢᴼⁿ= LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

    #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

    #h5open(tests_dir*tests_data_file_name,"cw") do fid
        #γ_group = open_group(fid, γᵢ) # open gamma coupling group
        #init_state_group = create_group(γ_group, state) # create initial state group
        #init_state_group["Fidelity_SB"] = convert.(Float64, Fˢᵖⁱⁿᵇᴼˢᴼⁿ)
        #init_state_group["F"] = convert.(Float64, F)
        #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
        #init_state_group["bloch_sid_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["bloch_sim_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        #init_state_group["time"] = tᵗˢᵗ
    #end
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Mimimal fidelity for 0.25133: 0.9993938976827987
Median fidelity for 0.25133: 0.9999711923879758
